#Imports, Data Downloading and Preprocessing

In [1]:
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms
import torch

import numpy as np

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA jest dostępna. Używam GPU.")
else:
    device = torch.device("cpu")
    print("CUDA nie jest dostępna. Używam CPU.")

CUDA jest dostępna. Używam GPU.


In [3]:
transform = transforms.Compose([transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:11<00:00, 834412.39it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 291011.09it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 839585.58it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4556452.71it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [4]:
flattened_images = []
labels = []

for image, label in mnist_train:
    flattened_image = image.flatten()
    flattened_images.append(flattened_image)
    labels.append(label)

tensor_train_imgs = torch.stack(flattened_images).to(device)
tensor_train_labels = torch.tensor(labels).unsqueeze(1).to(device)

In [5]:
flattened_images = []
labels = []

for image, label in mnist_test:
    flattened_image = image.flatten()
    flattened_images.append(flattened_image)
    labels.append(label)

tensor_test_imgs = torch.stack(flattened_images).to(device)
tensor_test_labels = torch.tensor(labels).unsqueeze(1).to(device)

#KNN algorithm

In [13]:
def euclidean_distances(test_features, train_features):
    test_squared_norms = torch.sum(test_features ** 2, dim=1).unsqueeze(1)
    train_squared_norms = torch.sum(train_features ** 2, dim=1).unsqueeze(0)

    dot_product = torch.mm(test_features, train_features.t())

    dists_squared = test_squared_norms + train_squared_norms - 2 * dot_product

    dists_squared = torch.clamp(dists_squared, min=0.0)

    dists = torch.sqrt(dists_squared)

    return dists

In [14]:
def get_distances(test_features, train_features):
    return euclidean_distances(test_features, train_features)

In [15]:
def knn(test_features, train_features, train_labels, k=3):
    dists = get_distances(test_features, train_features)

    _, knn_indices = torch.topk(dists, k, largest=False, dim=1)

    nearest_neighbours_matrix = train_labels[knn_indices].squeeze()

    # in single test_example case we have to add one dimension
    if len(nearest_neighbours_matrix.size()) < 2:
        nearest_neighbours_matrix = nearest_neighbours_matrix.unsqueeze(0)

    batch_size, num_elements = nearest_neighbours_matrix.shape

    counts = torch.zeros(batch_size, 10, dtype=torch.int).to(device)

    counts.scatter_add_(dim=1, index=nearest_neighbours_matrix, src=torch.ones_like(nearest_neighbours_matrix, dtype=torch.int))

    most_frequent = torch.argmax(counts, dim=1)

    return most_frequent

In [16]:
def single_example_knn(test_feature, train_features, train_labels, k=3):
    return knn(test_feature.unsqueeze(0), train_features, train_labels, k)

In [17]:
train_features = tensor_train_imgs
test_features = tensor_test_imgs
train_labels = tensor_train_labels
test_labels = tensor_test_labels

k = 3

mnist_guesses = knn(test_features, train_features, train_labels, k)
mnist_guess = single_example_knn(test_features[0],train_features, train_labels, k)

TypeError: topk(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
X, y = make_moons(n_samples=1000, noise=1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train).to(device)
X_test = torch.tensor(X_test).to(device)
y_train = torch.tensor(y_train).to(device)
y_test = torch.tensor(y_test).to(device)

k = 3

moons_guesses = knn(X_test, X_train, y_train, k)
moons_guess = single_example_knn(X_test[0],X_train, y_train, k)

#Algorithm Evaluation

In [ ]:
accuracy = torch.sum((test_labels.flatten()==mnist_guesses).int())/test_features.shape[0]

print(round(accuracy.item(),4))

print(f"\nSingle guess: {mnist_guess.item()}")
print(f"True label for single guess: {test_labels[0].item()}")

0.9705

Single guess: 7
True label for single guess: 7


In [ ]:
accuracy = torch.sum((y_test.flatten()==moons_guesses).int())/y_test.shape[0]

print(round(accuracy.item(),4))

print(f"\nSingle guess: {moons_guess.item()}")
print(f"True label for single guess: {y_test[0].item()}")

0.61

Single guess: 1
True label for single guess: 1


##Sklearn knn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sklearn_knn = KNeighborsClassifier(n_neighbors=3)  # You can adjust the number of neighbors
sklearn_knn.fit(np.array(train_features.cpu()), np.array(train_labels.cpu()))

y_pred = sklearn_knn.predict(np.array(test_features.cpu()))

accuracy = accuracy_score(np.array(test_labels.cpu()), y_pred)
print(f"Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Accuracy: 0.9705


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sklearn_knn = KNeighborsClassifier(n_neighbors=3)  # You can adjust the number of neighbors
sklearn_knn.fit(X_train, y_train)

y_pred = sklearn_knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6100
